### Importación de datos



In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tiendas = [tienda, tienda2, tienda3, tienda4] # Se crea una lista con los dataframes de cada tienda
# Se concatenan los dataframes de cada tienda en uno solo e ignorando los índices
# Se utiliza la función pd.concat para unir los dataframes en uno solo
db_tiendas = pd.concat(tiendas, ignore_index=True)

#db_tiendas.iloc[len(db_tiendas[db_tiendas["Lugar de Compra"]=="Manizales"]), :] # Se muestran las últimas 5 filas del dataframe

db_tiendas


,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359
...,...,...,...,...,...,...,...,...,...,...,...,...
9430,Mesa de comedor,Muebles,227700.0,12600.0,12/02/2020,Camila Rivera,Medellín,5,Tarjeta de crédito,1,6.25184,-75.56359
9431,Silla de oficina,Muebles,342000.0,18600.0,17/05/2022,Felipe Santos,Bucaramanga,5,Tarjeta de crédito,3,7.12539,-73.11980
9432,Vaso térmico,Deportes y diversión,36700.0,2400.0,03/05/2022,Camila Rivera,Bogotá,5,Tarjeta de crédito,1,4.60971,-74.08175
9433,Set de ollas,Artículos para el hogar,151800.0,8300.0,15/06/2020,Blanca Ramirez,Cali,5,Tarjeta de crédito,1,3.43722,-76.52250


# 1. Análisis de facturación



In [2]:
"""Se crea un grupo con Lugar de Compra, sumando los precios, el costo de envio y calculando un promedio de la calificación por Lugar de Compra (Tienda), 
para tener una vision general del comportamiento de los datos para cada tienda"""

utilidades = db_tiendas.groupby(["Lugar de Compra", "lat", "lon"]).agg({
    "Precio": "sum",
    "Costo de envío": "sum",
    "Calificación": "mean"
})

# Se crea una nueva columna para obtener la ganancia tras restar el precio por el costo de envío
utilidades["Ganancia"] = utilidades["Precio"] - utilidades["Costo de envío"]

utilidades["p_ganancia/precio"] = (utilidades["Ganancia"] / utilidades["Precio"])*100 # Se crea una nueva columna para obtener el porcentaje de precio por ganancia

utilidades["p_costo/precio"] = (utilidades["Costo de envío"] / utilidades["Precio"])*100 # Se crea una nueva columna para obtener el porcentaje de precio por costo de envío

utilidades.sort_values(by="Ganancia", ascending=False, inplace=True) # Se ordena el dataframe por la columna Ganancia de mayor a menor
utilidades.reset_index(inplace=True) # Se reinicia el índice para que quede más ordenado
utilidades

# Se puede observar que el porcentaje de los costos es relativamente similar para todas las tiendas. Se encuentran alrededor del 5,2% y el 5,4% del ingreso total.
# Dado esto tambien podemos observar que las ganancias son relativamente similares, oscilando entre el 94,6% y el 94,8% del ingreso total.
# Tambien podemos ver que las tiendas que mayores ingresos tienen son Bogotá y Medellín, mientras que las que menos ingresos en los 4 años fueron Valledupar y Inírida.

,Lugar de Compra,lat,lon,Precio,Costo de envío,Calificación,Ganancia,p_ganancia/precio,p_costo/precio
0,Bogotá,4.60971,-74.08175,1.814858e+09,96730000.0,4.113619,1.718128e+09,94.670107,5.329893
1,Medellín,6.25184,-75.56359,1.096669e+09,58502600.0,3.926534,1.038167e+09,94.665429,5.334571
2,Cali,3.43722,-76.52250,5.065261e+08,26913900.0,4.046673,4.796122e+08,94.686572,5.313428
3,Pereira,4.81333,-75.69611,2.259273e+08,12000300.0,4.013540,2.139270e+08,94.688424,5.311576
4,Barranquilla,10.96854,-74.78132,1.320405e+08,7034600.0,3.803226,1.250059e+08,94.672392,5.327608
5,Cartagena,10.39972,-75.51444,1.023767e+08,5478500.0,4.020661,9.689820e+07,94.648685,5.351315
6,Cúcuta,7.89391,-72.50782,9.936910e+07,5319100.0,3.940860,9.405000e+07,94.647129,5.352871
7,Bucaramanga,7.12539,-73.11980,9.057330e+07,4849000.0,3.977901,8.572430e+07,94.646325,5.353675
8,Riohacha,11.54444,-72.90722,7.387970e+07,3932800.0,3.895706,6.994690e+07,94.676752,5.323248
9,Leticia,-4.21528,-69.94056,6.714990e+07,3595100.0,3.922535,6.355480e+07,94.646157,5.353843


### Ganancias (ingresos - costos) por tienda al año

In [3]:
# Analizando el comportamiento de las ganancias por tienda y por año
# Se crea un nuevo dataFrame que contenga los Precios, los costos de envío y la calificación por cada tienda y por cada año
# Se establece el formato fecha para la columna "Fecha de Compra" y se extraen los días, meses y años para crear nuevas columnas, con los datos antes mencionados
db_tiendas["Fecha de Compra"] = pd.to_datetime(db_tiendas["Fecha de Compra"], format="%d/%m/%Y")
db_tiendas["dia"] = db_tiendas["Fecha de Compra"].dt.day
db_tiendas["mes"] = db_tiendas["Fecha de Compra"].dt.month
db_tiendas["año"] = db_tiendas["Fecha de Compra"].dt.year

# Se crea un nuevo dataframe agrupandos por Lugar de compra y por año, sumando las ganancias de cada mes

comportamiento_al_anio = db_tiendas.groupby(["Lugar de Compra", "año"]).agg({
    "Precio": "sum",
    "Costo de envío": "sum",
    "Calificación": "mean"
})

comportamiento_al_anio



Precio  Costo de envío  Calificación
Lugar de Compra año                                           
Armenia         2020   5382100.0        286000.0      3.111111
                2021   5500300.0        286300.0      3.250000
                2022   1135400.0         55400.0      4.000000
                2023    225100.0         12600.0      5.000000
Barranquilla    2020  48804300.0       2584300.0      3.828283
...                          ...             ...           ...
Valledupar      2023    238600.0         12900.0      5.000000
Villavicencio   2020   3482700.0        185800.0      3.750000
                2021   2812400.0        148700.0      4.000000
                2022   6916100.0        386200.0      3.692308
                2023    122400.0          7200.0      5.000000

[75 rows x 3 columns]

In [4]:
# Se crea una nueva columna para obtener la ganancia tras restar el precio por el costo de envío, para el año en cuestión

comportamiento_al_anio["Ganancia"]= comportamiento_al_anio["Precio"] - comportamiento_al_anio["Costo de envío"]
comportamiento_al_anio = comportamiento_al_anio.reset_index()

comportamiento_al_anio


,Lugar de Compra,año,Precio,Costo de envío,Calificación,Ganancia
0,Armenia,2020,5382100.0,286000.0,3.111111,5096100.0
1,Armenia,2021,5500300.0,286300.0,3.250000,5214000.0
2,Armenia,2022,1135400.0,55400.0,4.000000,1080000.0
3,Armenia,2023,225100.0,12600.0,5.000000,212500.0
4,Barranquilla,2020,48804300.0,2584300.0,3.828283,46220000.0
...,...,...,...,...,...,...
70,Valledupar,2023,238600.0,12900.0,5.000000,225700.0
71,Villavicencio,2020,3482700.0,185800.0,3.750000,3296900.0
72,Villavicencio,2021,2812400.0,148700.0,4.000000,2663700.0
73,Villavicencio,2022,6916100.0,386200.0,3.692308,6529900.0


In [5]:
# Se realiza un pivot (transporte de los datos) para que las columnas sean los años y las filas los lugares de compra, para facilitar la visibilidad de los datos. Se llenan los valores nulos con 0
df_pivot_tendencia = comportamiento_al_anio.pivot(index='Lugar de Compra', columns='año', values='Ganancia').fillna(0)

# Se crea una nueva columna para sumar las ganancias de cada año y se ordena de mayor a menor
df_pivot_tendencia["ganancias_totales"] = df_pivot_tendencia[2020] + df_pivot_tendencia[2021] + df_pivot_tendencia[2022] + df_pivot_tendencia[2023]

# Se ordena el dataframe por la columna "ventas_totales" de mayor a menor
df_pivot_tendencia = df_pivot_tendencia.sort_values("ganancias_totales", ascending=False)

df_pivot_tendencia = df_pivot_tendencia.rename(columns={2020: "Ganancias 2020", 2021: "Ganancias 2021", 2022: "Ganancias 2022", 2023: "Ganancias 2023"}) # Se renombran las columnas para que tengan un nombre más claro

# Se resetean los índices
df_pivot_tendencia = df_pivot_tendencia.reset_index()



df_pivot_tendencia["ganancias_totales"] = df_pivot_tendencia.apply(lambda fila: f"{fila["ganancias_totales"]:,.0f}", axis = 1) # Se formatea la columna "ventas_totales" para que tenga separador de miles y no decimales

df_pivot_tendencia

# Aqui observamos que muchas de las tiendas han tenido una disminucion en las ganancias a lo largo de los años. Y observamos que la sucursal de Manizales no tuvo ninguna ganancia en 2023

año,Lugar de Compra,Ganancias 2020,Ganancias 2021,Ganancias 2022,Ganancias 2023,ganancias_totales
0,Bogotá,540584100.0,553351200.0,507248100.0,116944900.0,"1,718,128,300"
1,Medellín,291605900.0,368167000.0,310684700.0,67709000.0,"1,038,166,600"
2,Cali,148962600.0,133712700.0,150304300.0,46632600.0,"479,612,200"
3,Pereira,60152400.0,70802100.0,61533500.0,21439000.0,"213,927,000"
4,Barranquilla,46220000.0,39388900.0,31053000.0,8344000.0,"125,005,900"
5,Cartagena,23353100.0,34116800.0,37805400.0,1622900.0,"96,898,200"
6,Cúcuta,34522100.0,24833400.0,26932300.0,7762200.0,"94,050,000"
7,Bucaramanga,22345700.0,29694200.0,29017200.0,4667200.0,"85,724,300"
8,Riohacha,25614700.0,19418500.0,19668500.0,5245200.0,"69,946,900"
9,Leticia,23442200.0,20489900.0,14820400.0,4802300.0,"63,554,800"


In [6]:
# Si filtramos a Manizales en la base de datos completa podemos ver que no tuvo ventas en 2023.

db_tiendas[(db_tiendas["Lugar de Compra"]=="Manizales") & (db_tiendas["año"]==2023)].reset_index()

,index,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon,dia,mes,año


### Ingresos por tienda al año

In [7]:
# Se realiza un pivot para que las columnas sean los años y las filas los lugares de compra, y se llenan los valores con el Precio, a cada valor nulo de rellena con cero
df_pivot_ingresos = comportamiento_al_anio.pivot(index='Lugar de Compra', columns='año', values='Precio').fillna(0)
df_pivot_ingresos["ingresos_totales"] = df_pivot_ingresos[2020] + df_pivot_ingresos[2021] + df_pivot_ingresos[2022] + df_pivot_ingresos[2023]
df_pivot_ingresos = df_pivot_ingresos.sort_values("ingresos_totales", ascending=False)  
df_pivot_ingresos = df_pivot_ingresos.reset_index()
# df_pivot_ingresos["ingresos_totales"] = df_pivot_ingresos.apply(lambda fila: f"{fila["ingresos_totales"]:,.0f}", axis = 1) # Se formatea la columna "ventas_totales" para que tenga separador de miles y no decimales

df_pivot_ingresos = df_pivot_ingresos.rename(columns={2020:"Ingresos 2020", 2021:"Ingresos 2021", 2022:"Ingresos 2022", 2023:"Ingresos 2023"})
df_pivot_ingresos

año,Lugar de Compra,Ingresos 2020,Ingresos 2021,Ingresos 2022,Ingresos 2023,ingresos_totales
0,Bogotá,570984200.0,584437600.0,535839900.0,123596600.0,1.814858e+09
1,Medellín,308032200.0,388914500.0,328261800.0,71460700.0,1.096669e+09
2,Cali,157377000.0,141193300.0,158714600.0,49241200.0,5.065261e+08
3,Pereira,63523100.0,74769700.0,65015000.0,22619500.0,2.259273e+08
4,Barranquilla,48804300.0,41620300.0,32795400.0,8820500.0,1.320405e+08
5,Cartagena,24662000.0,36051700.0,39951200.0,1711800.0,1.023767e+08
6,Cúcuta,36472600.0,26230800.0,28431500.0,8234200.0,9.936910e+07
7,Bucaramanga,23624700.0,31366600.0,30656300.0,4925700.0,9.057330e+07
8,Riohacha,27053600.0,20490800.0,20793800.0,5541500.0,7.387970e+07
9,Leticia,24758400.0,21665800.0,15653200.0,5072500.0,6.714990e+07


### Costos (de envio) por tienda al año

In [8]:
# Se crea un nuevo dataframe agrupandos por Lugar de compra (filas) y por año (columnas), sumando los costos de envío de cada mes
df_pivot_envios = comportamiento_al_anio.pivot(index='Lugar de Compra', columns='año', values='Costo de envío').fillna(0)
df_pivot_envios["costo_envios_totales"] = df_pivot_envios[2020] + df_pivot_envios[2021] + df_pivot_envios[2022] + df_pivot_envios[2023]
df_pivot_envios = df_pivot_envios.sort_values("costo_envios_totales", ascending=False)
df_pivot_envios = df_pivot_envios.rename(columns={2020:"Costo Envios 2020", 2021:"Costo Envios 2021", 2022:"Costo Envios 2022", 2023:"Costo Envios 2023"}) # Se renombran las columnas para que tengan un nombre más claro
df_pivot_envios = df_pivot_envios.reset_index()
# df_pivot_envios["envios_totales"] = df_pivot_envios.apply(lambda fila: f"{fila["envios_totales"]:,.0f}", axis = 1) # Se formatea la columna "ventas_totales" para que tenga separador de miles y no decimales
df_pivot_envios

año,Lugar de Compra,Costo Envios 2020,Costo Envios 2021,Costo Envios 2022,Costo Envios 2023,costo_envios_totales
0,Bogotá,30400100.0,31086400.0,28591800.0,6651700.0,96730000.0
1,Medellín,16426300.0,20747500.0,17577100.0,3751700.0,58502600.0
2,Cali,8414400.0,7480600.0,8410300.0,2608600.0,26913900.0
3,Pereira,3370700.0,3967600.0,3481500.0,1180500.0,12000300.0
4,Barranquilla,2584300.0,2231400.0,1742400.0,476500.0,7034600.0
5,Cartagena,1308900.0,1934900.0,2145800.0,88900.0,5478500.0
6,Cúcuta,1950500.0,1397400.0,1499200.0,472000.0,5319100.0
7,Bucaramanga,1279000.0,1672400.0,1639100.0,258500.0,4849000.0
8,Riohacha,1438900.0,1072300.0,1125300.0,296300.0,3932800.0
9,Leticia,1316200.0,1175900.0,832800.0,270200.0,3595100.0


In [9]:
#df_pivot_tendencia = df_pivot_tendencia.drop(columns=["desviaciones 2021", "desviaciones 2022", "desviaciones 2023"]) # Se eliminan las columnas de los años para dejar solo la columna de ventas totales y la de lugar de compra

### Variaciones de ganancias en comparacion con año base (2020)

In [10]:
# Se comparan las ganancias de cada año, con el año 2020 como base

df_pivot_tendencia["var_ganancias 2021"] = (((df_pivot_tendencia["Ganancias 2021"] - df_pivot_tendencia["Ganancias 2020"]) / df_pivot_tendencia["Ganancias 2020"]) * 100).round(2) # Se redondea a 2 decimales
df_pivot_tendencia["var_ganancias 2022"] = (((df_pivot_tendencia["Ganancias 2022"] - df_pivot_tendencia["Ganancias 2020"]) / df_pivot_tendencia["Ganancias 2020"]) * 100).round(2) # Se redondea a 2 decimales
df_pivot_tendencia["var_ganancias 2023"] = (((df_pivot_tendencia["Ganancias 2023"] - df_pivot_tendencia["Ganancias 2020"]) / df_pivot_tendencia["Ganancias 2020"]) * 100).round(2) # Se redondea a 2 decimales


# Se calcula el promedio de las variaciones en las ganancias de cada año respecto al año 2020
df_pivot_tendencia["promedio_variaciones"] = (df_pivot_tendencia["var_ganancias 2021"] + df_pivot_tendencia["var_ganancias 2022"] + df_pivot_tendencia["var_ganancias 2023"]) / 3 
# df_pivot_tendencia[2020] = df_pivot_tendencia.apply(lambda fila: f"{fila[2020]:,.0f}", axis = 1) # Se formatea la columna "2020" para que tenga separador de miles y no decimales
# df_pivot_tendencia[2021] = df_pivot_tendencia.apply(lambda fila: f"{fila[2021]:,.0f}", axis = 1) # Se formatea la columna "2021" para que tenga separador de miles y no decimales
# df_pivot_tendencia[2022] = df_pivot_tendencia.apply(lambda fila: f"{fila[2022]:,.0f}", axis = 1) # Se formatea la columna "2022" para que tenga separador de miles y no decimales
# df_pivot_tendencia[2023] = df_pivot_tendencia.apply(lambda fila: f"{fila[2023]:,.0f}", axis = 1) # Se formatea la columna "2023" para que tenga separador de miles y no decimales
df_pivot_tendencia.sort_values("promedio_variaciones", ascending=False, inplace=True) # Se ordena el dataframe por la columna "desviaciones 2023" de mayor a menor


df_pivot_tendencia[["Lugar de Compra","var_ganancias 2021", "var_ganancias 2022", "var_ganancias 2023"]]

# Se observa las tiendas que ha tenido una mayor variacion positiva en sus ganancias en comparacion con el año 2020 es Soacha.
# Tambien se observa que la tienda con mayor variacion negativa es Neiva

año,Lugar de Compra,var_ganancias 2021,var_ganancias 2022,var_ganancias 2023
15,Soacha,227.98,366.82,-34.26
17,Inírida,36.33,209.48,-49.84
13,Manizales,149.81,81.22,-100.00
10,Santa Marta,15.63,109.62,-54.53
5,Cartagena,46.09,61.89,-93.05
7,Bucaramanga,32.89,29.86,-79.11
14,Villavicencio,-19.21,98.06,-96.51
1,Medellín,26.25,6.54,-76.78
3,Pereira,17.70,2.30,-64.36
2,Cali,-10.24,0.90,-68.70


### Porcentaje de utilidades (ganancias), en comparacion con los ingresos totales del año

In [11]:
margen = df_pivot_tendencia.merge(df_pivot_ingresos, on="Lugar de Compra", how="inner") # Se unen los dataframes de ingresos y ganancias por la columna "Lugar de Compra"
margen["margen 2020"] = (margen["Ganancias 2020"] / margen["Ingresos 2020"])*100 # Se calcula el margen de ganancias sobre ingresos
margen["margen 2021"] = (margen["Ganancias 2021"] / margen["Ingresos 2021"])*100 # Se calcula el margen de ganancias sobre ingresos
margen["margen 2022"] = (margen["Ganancias 2022"] / margen["Ingresos 2022"])*100 # Se calcula el margen de ganancias sobre ingresos
margen["margen 2023"] = (margen["Ganancias 2023"] / margen["Ingresos 2023"])*100 # Se calcula el margen de ganancias sobre ingresos

margen[["Lugar de Compra","margen 2020", "margen 2021", "margen 2022", "margen 2023"]]

# Se observa que el marge por año es relativamente similar al obtenido con datos generales.

año,Lugar de Compra,margen 2020,margen 2021,margen 2022,margen 2023
0,Soacha,94.879212,94.505912,94.678878,94.626972
1,Inírida,94.877095,94.397457,94.756902,94.948454
2,Manizales,94.711446,94.689489,94.762654,NaN
3,Santa Marta,94.629297,94.595402,94.706671,94.754303
4,Cartagena,94.692645,94.632985,94.628947,94.806636
5,Bucaramanga,94.586175,94.668214,94.653301,94.752015
6,Villavicencio,94.665059,94.712701,94.415928,94.117647
7,Medellín,94.667343,94.665280,94.645402,94.749981
8,Pereira,94.693741,94.693572,94.645082,94.781052
9,Cali,94.653348,94.701873,94.700992,94.702404


#3. Calificación promedio de la tienda


### Calificaciones promedio por tienda al año

In [12]:
# Se crea un nuevo dataframe agrupando por Lugar de compra y por año, promediand las calificaciones de cada año
calificaciones = db_tiendas.groupby(["Lugar de Compra", "año"]).agg({
    "Calificación": "mean",
})
calificaciones = calificaciones.reset_index()


df_pivot_calificaciones = calificaciones.pivot(index='Lugar de Compra', columns='año', values='Calificación').fillna(0)
df_pivot_calificaciones["calificacion_total"] = (df_pivot_calificaciones[2020] + df_pivot_calificaciones[2021] + df_pivot_calificaciones[2022] + df_pivot_calificaciones[2023])/4

df_pivot_calificaciones = df_pivot_calificaciones.rename(columns={2020:"Calificacion 2020", 2021:"Calificacion 2021", 2022:"Calificacion 2022", 2023:"Calificacion 2023"}) # Se renombran las columnas para que tengan un nombre más claro

df_pivot_calificaciones = df_pivot_calificaciones.sort_values("calificacion_total", ascending=False)

df_pivot_calificaciones

año,Calificacion 2020,Calificacion 2021,Calificacion 2022,Calificacion 2023,calificacion_total
Lugar de Compra,,,,,
Valledupar,4.400000,3.250000,4.250000,5.000000,4.225000
Bogotá,4.123762,4.077607,4.136439,4.130137,4.116986
Villavicencio,3.750000,4.000000,3.692308,5.000000,4.110577
Cali,4.126667,3.923588,4.057325,4.152174,4.064938
Pasto,3.766667,4.533333,4.333333,3.428571,4.015476
Soacha,4.250000,4.666667,3.125000,4.000000,4.010417
Neiva,4.000000,4.000000,3.000000,5.000000,4.000000
Pereira,4.068493,3.943038,4.061350,3.920000,3.998220
Bucaramanga,4.060000,3.962264,3.903226,4.062500,3.996997


In [13]:
# Se unen los dataframes de tendencia (ganancias) y calificaciones por la fila "Lugar de Compra" para tener una tabla con las ganancias y las calificaciones de cada tienda
df_full = df_pivot_tendencia.merge(df_pivot_calificaciones, on="Lugar de Compra", how="inner") 

# Se calcula el porcentaje de ganancias que aporta cada tienda sobre el total de ganancias
df_full["P_ganancias 2020"] = (df_full["Ganancias 2020"] / df_full["Ganancias 2020"].sum()) * 100 
df_full["P_ganancias 2021"] = (df_full["Ganancias 2021"] / df_full["Ganancias 2021"].sum()) * 100 # Se calcula el porcentaje de ganancias de cada tienda sobre el total de ganancias
df_full["P_ganancias 2022"] = (df_full["Ganancias 2022"] / df_full["Ganancias 2022"].sum()) * 100 # Se calcula el porcentaje de ganancias de cada tienda sobre el total de ganancias
df_full["P_ganancias 2023"] = (df_full["Ganancias 2023"] / df_full["Ganancias 2023"].sum()) * 100 # Se calcula el porcentaje de ganancias de cada tienda sobre el total de ganancias
#df_full["P_ganancias totales"] = df_full["ganancias_totales"] / df_full["ganancias_totales"].sum() * 100 # Se calcula el porcentaje de ganancias de cada tienda sobre el total de ganancias
df_full = df_full.reset_index() # Se reinicia el índice para que quede más ordenado
df_full = df_full.sort_values("P_ganancias 2023", ascending=False) # Se ordena el dataframe por la columna "P_ganancias 2023" de mayor a menor
df_full = df_full.reset_index() # Se reinicia el índice para que quede más ordenado

df_full = df_full.drop(columns=["level_0"]) 

In [14]:
df_full[["Lugar de Compra","P_ganancias 2020", "P_ganancias 2021", "P_ganancias 2022", "P_ganancias 2023"]]

año,Lugar de Compra,P_ganancias 2020,P_ganancias 2021,P_ganancias 2022,P_ganancias 2023
0,Bogotá,42.550774,41.040869,40.366384,39.834883
1,Medellín,22.953055,27.306155,24.724031,23.063683
2,Cali,11.725232,9.917183,11.961092,15.884439
3,Pereira,4.734751,5.251240,4.896785,7.302756
4,Barranquilla,3.638096,2.921390,2.471172,2.842213
5,Cúcuta,2.717324,1.841840,2.143250,2.644034
6,Riohacha,2.016199,1.440228,1.565203,1.786670
7,Santa Marta,0.840587,0.915839,1.781467,1.653892
8,Leticia,1.845196,1.519692,1.179395,1.635805
9,Bucaramanga,1.758888,2.202355,2.309165,1.589786


In [15]:
# Ordenar el DataFrame por "Ganancias 2023" de mayor a menor
df_full = df_full.sort_values("P_ganancias 2023", ascending=False)

# Calcular la ganancia acumulada para el año 2023
df_full["ganancia acum 2023"] = df_full["P_ganancias 2023"].cumsum()

df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 19 non-null     int64  
 1   Lugar de Compra       19 non-null     object 
 2   Ganancias 2020        19 non-null     float64
 3   Ganancias 2021        19 non-null     float64
 4   Ganancias 2022        19 non-null     float64
 5   Ganancias 2023        19 non-null     float64
 6   ganancias_totales     19 non-null     object 
 7   var_ganancias 2021    19 non-null     float64
 8   var_ganancias 2022    19 non-null     float64
 9   var_ganancias 2023    19 non-null     float64
 10  promedio_variaciones  19 non-null     float64
 11  Calificacion 2020     19 non-null     float64
 12  Calificacion 2021     19 non-null     float64
 13  Calificacion 2022     19 non-null     float64
 14  Calificacion 2023     19 non-null     float64
 15  calificacion_total    19 

In [16]:
df_full[["Lugar de Compra", "ganancia acum 2023","Ganancias 2023", "P_ganancias 2023", "Calificacion 2020", "Calificacion 2021", "Calificacion 2022", "Calificacion 2023", "var_ganancias 2021", "var_ganancias 2022", "var_ganancias 2023",]] # Se muestran las primeras 10 filas del dataframe con las columnas seleccionadas

año,Lugar de Compra,ganancia acum 2023,Ganancias 2023,P_ganancias 2023,Calificacion 2020,Calificacion 2021,Calificacion 2022,Calificacion 2023,var_ganancias 2021,var_ganancias 2022,var_ganancias 2023
0,Bogotá,39.834883,116944900.0,39.834883,4.123762,4.077607,4.136439,4.130137,2.36,-6.17,-78.37
1,Medellín,62.898566,67709000.0,23.063683,3.930636,3.885827,3.954942,3.976744,26.25,6.54,-76.78
2,Cali,78.783006,46632600.0,15.884439,4.126667,3.923588,4.057325,4.152174,-10.24,0.90,-68.70
3,Pereira,86.085762,21439000.0,7.302756,4.068493,3.943038,4.061350,3.920000,17.70,2.30,-64.36
4,Barranquilla,88.927974,8344000.0,2.842213,3.828283,3.841584,3.741176,3.760000,-14.78,-32.81,-81.95
5,Cúcuta,91.572009,7762200.0,2.644034,3.983871,3.903226,4.140000,3.083333,-28.07,-21.99,-77.52
6,Riohacha,93.358678,5245200.0,1.786670,3.964286,3.666667,4.000000,4.083333,-24.19,-23.21,-79.52
7,Santa Marta,95.012571,4855400.0,1.653892,3.500000,3.363636,3.788462,3.866667,15.63,109.62,-54.53
8,Leticia,96.648376,4802300.0,1.635805,3.977273,3.625000,4.282051,3.727273,-12.59,-36.78,-79.51
9,Bucaramanga,98.238162,4667200.0,1.589786,4.060000,3.962264,3.903226,4.062500,32.89,29.86,-79.11


Las sucursales que generan el 90% de las ganancias son Bogotá, Medellín, Cali, Pereira, Barranquilla y Cúcuta (alrrededor del 91% de las ganancias totales). Los productos mas vendidos corresponden a electronicos y muebles.

In [17]:
print(f"Ganancias totales 2020: {df_full["Ganancias 2020"].sum():,.0f}\nGanancias totales 2023: {df_full["Ganancias 2023"].sum():,.0f}\nPorcentaje de variacion total:{((df_full["Ganancias 2023"].sum()-df_full["Ganancias 2020"].sum())/df_full["Ganancias 2020"].sum())*100} ")

Ganancias totales 2020: 1,270,444,800
Ganancias totales 2023: 293,574,100
Porcentaje de variacion total:-76.89202238460105 


In [18]:
numero_de_ventas_al_anio = db_tiendas.groupby(["Lugar de Compra","año"]).agg({
    "año": "count",
})

numero_de_ventas_al_anio = numero_de_ventas_al_anio.rename(columns={"año":"Numero de ventas"})
numero_de_ventas_al_anio = numero_de_ventas_al_anio.reset_index() # Se reinicia el índice para que quede más ordenado
pivot_numero_de_ventas_al_anio = numero_de_ventas_al_anio.pivot(index="Lugar de Compra", columns="año", values="Numero de ventas").fillna(0) 


precio_promedio_al_anio = db_tiendas.groupby(["Lugar de Compra","año"]).agg({
    "Precio": "sum",
})
precio_promedio_al_anio = precio_promedio_al_anio.rename(columns={"Precio":"Precio promedio"})
precio_promedio_al_anio = precio_promedio_al_anio.reset_index() # Se reinicia el índice para que quede más ordenado
pivot_precio_promedio_al_anio = precio_promedio_al_anio.pivot(index="Lugar de Compra", columns="año", values="Precio promedio").fillna(0)
pivot_precio_promedio_al_anio



año,2020,2021,2022,2023
Lugar de Compra,,,,
Armenia,5382100.0,5500300.0,1135400.0,225100.0
Barranquilla,48804300.0,41620300.0,32795400.0,8820500.0
Bogotá,570984200.0,584437600.0,535839900.0,123596600.0
Bucaramanga,23624700.0,31366600.0,30656300.0,4925700.0
Cali,157377000.0,141193300.0,158714600.0,49241200.0
Cartagena,24662000.0,36051700.0,39951200.0,1711800.0
Cúcuta,36472600.0,26230800.0,28431500.0,8234200.0
Inírida,1354700.0,1856300.0,4197900.0,679000.0
Leticia,24758400.0,21665800.0,15653200.0,5072500.0


In [19]:
# pivote = pivot_precio_promedio_al_anio.merge(pivot_numero_de_ventas_al_anio, on="Lugar de Compra", how="inner")

# pivote["precio medio 2020"] = pivote["2020_x"] / pivote["2020_y"]
# pivote["precio medio 2021"] = pivote["2021_x"] / pivote["2021_y"]
# pivote["precio medio 2022"] = pivote["2022_x"] / pivote["2022_y"]
# pivote["precio medio 2023"] = pivote["2023_x"] / pivote["2023_y"]

# pivote_2 = pivote[["precio medio 2020", "precio medio 2021", "precio medio 2022", "precio medio 2023"]].copy()
# pivote_2

# 4. Productos más y menos vendidos

### Cantidad de ventas por producto

In [20]:
cant_ventas = db_tiendas.groupby(["Lugar de Compra","Producto", "año"]).agg({
    "Producto": "count"
})
cant_ventas = cant_ventas.rename(columns={"Producto": "Cantidad de Ventas"})

cant_ventas = cant_ventas.reset_index()
cant_ventas


,Lugar de Compra,Producto,año,Cantidad de Ventas
0,Armenia,Asistente virtual,2023,1
1,Armenia,Batería,2021,1
2,Armenia,Bloques de construcción,2020,1
3,Armenia,Bloques de construcción,2021,2
4,Armenia,Bloques de construcción,2023,1
...,...,...,...,...
1833,Villavicencio,Smart TV,2020,1
1834,Villavicencio,Smartwatch,2021,1
1835,Villavicencio,Smartwatch,2023,1
1836,Villavicencio,Vaso térmico,2020,1


In [21]:
# DataFrame con el producto más vendido por lugar de compra
producto_mas_vendido = cant_ventas.loc[cant_ventas.groupby("Lugar de Compra")["Cantidad de Ventas"].idxmax()].reset_index(drop=True)



producto_mas_vendido

,Lugar de Compra,Producto,año,Cantidad de Ventas
0,Armenia,Bloques de construcción,2021,2
1,Barranquilla,Ajedrez de madera,2021,6
2,Bogotá,Carrito de control remoto,2020,36
3,Bucaramanga,Juego de mesa,2022,6
4,Cali,Tablet ABXY,2021,14
5,Cartagena,Ajedrez de madera,2022,5
6,Cúcuta,Iphone 15,2020,5
7,Inírida,Bicicleta,2020,2
8,Leticia,Auriculares,2022,4
9,Manizales,Bicicleta,2021,2


In [22]:
# DataFrame con el producto menos vendido por lugar de compra
producto_menos_vendido = cant_ventas.loc[cant_ventas.groupby("Lugar de Compra")["Cantidad de Ventas"].idxmin()].reset_index(drop=True)

producto_menos_vendido

,Lugar de Compra,Producto,año,Cantidad de Ventas
0,Armenia,Asistente virtual,2023,1
1,Barranquilla,Asistente virtual,2020,1
2,Bogotá,Dashboards con Power BI,2023,1
3,Bucaramanga,Ajedrez de madera,2021,1
4,Cali,Asistente virtual,2023,1
5,Cartagena,Ajedrez de madera,2020,1
6,Cúcuta,Ajedrez de madera,2022,1
7,Inírida,Auriculares con micrófono,2020,1
8,Leticia,Armario,2021,1
9,Manizales,Armario,2020,1


In [23]:
# DataFrame con el producto más vendido por tienda al año
producto_mas_vendido_anio = db_tiendas.groupby(["Lugar de Compra", "año", "Producto"]).agg({
    "Producto": "count"
}).rename(columns={"Producto": "Cantidad de Ventas"}).reset_index()

producto_mas_vendido_anio = producto_mas_vendido_anio.loc[
    producto_mas_vendido_anio.groupby(["Lugar de Compra", "año"])["Cantidad de Ventas"].idxmax()
].reset_index(drop=True)

# DataFrame con el producto menos vendido por tienda al año
producto_menos_vendido_anio = db_tiendas.groupby(["Lugar de Compra", "año", "Producto"]).agg({
    "Producto": "count"
}).rename(columns={"Producto": "Cantidad de Ventas"}).reset_index()

producto_menos_vendido_anio = producto_menos_vendido_anio.loc[
    producto_menos_vendido_anio.groupby(["Lugar de Compra", "año"])["Cantidad de Ventas"].idxmin()
].reset_index(drop=True)

producto_mas_vendido_anio, producto_menos_vendido_anio

(   Lugar de Compra   año                 Producto  Cantidad de Ventas
 0          Armenia  2020              Tablet ABXY                   2
 1          Armenia  2021  Bloques de construcción                   2
 2          Armenia  2022               Cubertería                   1
 3          Armenia  2023        Asistente virtual                   1
 4     Barranquilla  2020      Modelado predictivo                   5
 ..             ...   ...                      ...                 ...
 70      Valledupar  2023         Silla de oficina                   1
 71   Villavicencio  2020               Microondas                   2
 72   Villavicencio  2021                 Cama box                   3
 73   Villavicencio  2022                Bicicleta                   2
 74   Villavicencio  2023               Smartwatch                   1
 
 [75 rows x 4 columns],
    Lugar de Compra   año                 Producto  Cantidad de Ventas
 0          Armenia  2020  Bloques de construcción 

In [24]:
# Agrupamos por Lugar de Compra, año y Categoría del Producto, y contamos las ventas
categorias_ventas = db_tiendas.groupby(["Lugar de Compra", "año", "Categoría del Producto"]).agg({
    "Producto": "count"
}).rename(columns={"Producto": "Cantidad de Ventas"}).reset_index()

# DataFrame con la categoría más vendida por tienda al año
categoria_mas_vendida_anio = categorias_ventas.loc[
    categorias_ventas.groupby(["Lugar de Compra", "año"])["Cantidad de Ventas"].idxmax()
].reset_index(drop=True)

# DataFrame con la categoría menos vendida por tienda al año
categoria_menos_vendida_anio = categorias_ventas.loc[
    categorias_ventas.groupby(["Lugar de Compra", "año"])["Cantidad de Ventas"].idxmin()
].reset_index(drop=True)

categoria_mas_vendida_anio

,Lugar de Compra,año,Categoría del Producto,Cantidad de Ventas
0,Armenia,2020,Electrónicos,3
1,Armenia,2021,Juguetes,4
2,Armenia,2022,Muebles,2
3,Armenia,2023,Electrónicos,1
4,Barranquilla,2020,Muebles,24
...,...,...,...,...
70,Valledupar,2023,Muebles,1
71,Villavicencio,2020,Electrodomésticos,2
72,Villavicencio,2021,Muebles,3
73,Villavicencio,2022,Deportes y diversión,3


In [25]:
categoria_mas_vendida_anio_pivot = categoria_mas_vendida_anio.pivot(index='Lugar de Compra', columns='año', values='Categoría del Producto').fillna(0)

In [26]:
categoria_mas_vendida_anio_pivot 

año,2020,2021,2022,2023
Lugar de Compra,,,,
Armenia,Electrónicos,Juguetes,Muebles,Electrónicos
Barranquilla,Muebles,Muebles,Muebles,Juguetes
Bogotá,Muebles,Muebles,Muebles,Muebles
Bucaramanga,Juguetes,Muebles,Juguetes,Muebles
Cali,Electrónicos,Electrónicos,Muebles,Electrónicos
Cartagena,Muebles,Electrónicos,Juguetes,Muebles
Cúcuta,Electrónicos,Muebles,Muebles,Electrónicos
Inírida,Deportes y diversión,Muebles,Electrodomésticos,Instrumentos musicales
Leticia,Muebles,Deportes y diversión,Electrónicos,Electrodomésticos


In [27]:
categoria_menos_vendida_anio_pivot = categoria_menos_vendida_anio.pivot(index='Lugar de Compra', columns='año', values='Categoría del Producto').fillna(0)
categoria_menos_vendida_anio_pivot

año,2020,2021,2022,2023
Lugar de Compra,,,,
Armenia,Libros,Deportes y diversión,Artículos para el hogar,Electrónicos
Barranquilla,Artículos para el hogar,Libros,Instrumentos musicales,Instrumentos musicales
Bogotá,Instrumentos musicales,Artículos para el hogar,Artículos para el hogar,Artículos para el hogar
Bucaramanga,Artículos para el hogar,Artículos para el hogar,Artículos para el hogar,Artículos para el hogar
Cali,Instrumentos musicales,Artículos para el hogar,Libros,Libros
Cartagena,Instrumentos musicales,Instrumentos musicales,Instrumentos musicales,Artículos para el hogar
Cúcuta,Libros,Artículos para el hogar,Instrumentos musicales,Artículos para el hogar
Inírida,Electrónicos,Deportes y diversión,Electrodomésticos,Instrumentos musicales
Leticia,Libros,Artículos para el hogar,Artículos para el hogar,Artículos para el hogar


In [28]:
df_armenia = db_tiendas.loc[(db_tiendas["Lugar de Compra"] == "Manizales") & (db_tiendas["año"]==2023)]
df_armenia

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon,dia,mes,año


In [29]:
%pip install dash

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import plotly.graph_objects as go


# Crear una lista de tiendas
tiendas_plot = pivot_precio_promedio_al_anio.index.tolist()
anos_plot = pivot_precio_promedio_al_anio.columns.tolist()

# Crear una figura inicial con la primera tienda
fig = go.Figure()

# Agregar trazas para cada tienda
for local in pivot_precio_promedio_al_anio.index:
    fig.add_trace(go.Bar(
        x=anos_plot,
        y=pivot_precio_promedio_al_anio.loc[local],  
        name=local,  # Nombre de la tienda
    ))

# Configurar el diseño del gráfico
fig.update_layout(
    title="Ingreso total por año y tienda",
    xaxis_title="Año",
    yaxis_title="Ingreso total",
    barmode= "group",
    legend_title="Tienda",
      
)

# Se puede visualizar que el costo por producto de forma general a traves de los años a aumentado significativamente en el ultimo año.
# Si bien como media cada producto vendido a sido mas costoso, la cantidad de articulos vendidos es significativamente menor, lo que puede indicar que el aumento de precios ha afectado la cantidad de ventas.

In [31]:
import plotly.graph_objects as go

tiendas_plot = pivot_numero_de_ventas_al_anio.index.tolist()
anos_plot_n = pivot_numero_de_ventas_al_anio.columns.tolist()

# Crear una figura inicial con la primera tienda
fig = go.Figure()

# Agregar trazas para cada tienda
for local in pivot_numero_de_ventas_al_anio.index:
    fig.add_trace(go.Bar(
        x=anos_plot_n,
        y=pivot_numero_de_ventas_al_anio.loc[local],  
        name=local,  # Nombre de la tienda
    ))

# Configurar el diseño del gráfico
fig.update_layout(
    title="Numero de ventas al año por tienda",
    xaxis_title="Año",
    yaxis_title="Numero de ventas",
    barmode= "group",
    legend_title="Tienda",
      
)

In [32]:
pivot_numero_de_ventas_al_anio

año,2020,2021,2022,2023
Lugar de Compra,,,,
Armenia,9.0,12.0,5.0,2.0
Barranquilla,99.0,101.0,85.0,25.0
Bogotá,1212.0,1237.0,1202.0,292.0
Bucaramanga,50.0,53.0,62.0,16.0
Cali,300.0,301.0,314.0,92.0
Cartagena,73.0,73.0,88.0,8.0
Cúcuta,62.0,62.0,50.0,12.0
Inírida,4.0,6.0,3.0,1.0
Leticia,44.0,48.0,39.0,11.0


In [33]:
# Ganancia por categoria de articulos
ganancia_por_categoria = db_tiendas.groupby(["Lugar de Compra","año" ,"Categoría del Producto"]).agg({
    "Precio": "sum",
    "Costo de envío": "sum",
    "Calificación": "mean"
})

ganancia_por_categoria["ganancia_neta"] = ganancia_por_categoria["Precio"] - ganancia_por_categoria["Costo de envío"]
ganancia_por_categoria = ganancia_por_categoria.reset_index()


In [34]:
pivot_ganancia_por_categoria = ganancia_por_categoria.pivot(index='Lugar de Compra', columns=['Categoría del Producto', "año"], values='ganancia_neta').fillna(0)
pivot_ganancia_por_categoria

Categoría del Producto,Artículos para el hogar,Electrónicos,Juguetes,Libros,Muebles,Deportes y diversión,Electrodomésticos,Electrónicos,Instrumentos musicales,Juguetes,...,Deportes y diversión,Electrónicos,Instrumentos musicales,Juguetes,Artículos para el hogar,Deportes y diversión,Electrodomésticos,Instrumentos musicales,Libros,Muebles
año,2020,2020,2020,2020,2020,2021,2021,2021,2021,2021,...,2022,2022,2022,2022,2023,2023,2023,2023,2023,2023
Lugar de Compra,,,,,,,,,,,,,,,,,,,,,
Armenia,99800.0,4407300.0,99400.0,49700.0,439900.0,48900.0,458200.0,3123800.0,573800.0,104200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Barranquilla,447300.0,17581800.0,313500.0,507800.0,8857900.0,1475300.0,10278800.0,12165900.0,5325900.0,635700.0,...,2773700.0,3955500.0,2686400.0,660400.0,137100.0,675100.0,4820600.0,732000.0,113300.0,900400.0
Bogotá,7360600.0,205061300.0,8333900.0,4328800.0,87080000.0,15688400.0,172259500.0,212123100.0,42127100.0,9298600.0,...,18610600.0,183124200.0,51834900.0,8654100.0,1453100.0,5330400.0,40474200.0,10208300.0,1398500.0,24609200.0
Bucaramanga,60100.0,9228700.0,843600.0,319200.0,3905300.0,753900.0,6344700.0,16121700.0,2258400.0,392300.0,...,394200.0,8540600.0,4362200.0,1329000.0,141000.0,952800.0,0.0,0.0,113100.0,1693700.0
Cali,1490100.0,56060200.0,2094000.0,1194400.0,19398600.0,3970100.0,39670600.0,53425200.0,5775900.0,2068500.0,...,4636200.0,56314600.0,10623400.0,2615200.0,450400.0,1042700.0,10336100.0,2554200.0,105400.0,6922600.0
Cartagena,546500.0,4893500.0,665200.0,434800.0,5658600.0,909000.0,9652400.0,18264100.0,1234500.0,897200.0,...,1072000.0,14729800.0,1695000.0,1181600.0,51100.0,446400.0,0.0,0.0,0.0,1029900.0
Cúcuta,277600.0,19258200.0,888100.0,88900.0,5933200.0,272800.0,12128300.0,5810100.0,568900.0,358600.0,...,249300.0,14783300.0,692500.0,263700.0,37500.0,56700.0,0.0,820600.0,44200.0,2090600.0
Inírida,0.0,89100.0,0.0,0.0,0.0,49200.0,670200.0,79900.0,682900.0,0.0,...,0.0,2294900.0,35500.0,0.0,0.0,0.0,0.0,644700.0,0.0,0.0


In [35]:
# Se calculo el porcentage de beneficios por categoria de articulos, para cada año en cada tienda
porcentaje_de_ganancia = pd.DataFrame()
porcentaje_de_ganancia["p_2020_art_hogar"] = (pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2020)]+ pivot_ganancia_por_categoria[("Juguetes", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2020)]+ pivot_ganancia_por_categoria[("Muebles", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]))*100

porcentaje_de_ganancia["p_2020_electronicos"] = (pivot_ganancia_por_categoria[("Electrónicos", 2020)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2020)]+ pivot_ganancia_por_categoria[("Juguetes", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2020)]+ pivot_ganancia_por_categoria[("Muebles", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]))*100

porcentaje_de_ganancia["p_2020_juguetes"] = (pivot_ganancia_por_categoria[("Juguetes", 2020)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2020)]+ pivot_ganancia_por_categoria[("Juguetes", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2020)]+ pivot_ganancia_por_categoria[("Muebles", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]))*100

porcentaje_de_ganancia["p_2020_libros"] = (pivot_ganancia_por_categoria[("Libros", 2020)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2020)]+ pivot_ganancia_por_categoria[("Juguetes", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2020)]+ pivot_ganancia_por_categoria[("Muebles", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]))*100

porcentaje_de_ganancia["p_2020_muebles"] = (pivot_ganancia_por_categoria[("Muebles", 2020)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2020)]+ pivot_ganancia_por_categoria[("Juguetes", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2020)]+ pivot_ganancia_por_categoria[("Muebles", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]))*100

porcentaje_de_ganancia["p_2020_deportes"] = (pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2020)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2020)]+ pivot_ganancia_por_categoria[("Juguetes", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2020)]+ pivot_ganancia_por_categoria[("Muebles", 2020)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2020)]))*100



porcentaje_de_ganancia["p_2021_art_hogar"] = (pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2021)]+ pivot_ganancia_por_categoria[("Juguetes", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2021)]+ pivot_ganancia_por_categoria[("Muebles", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]))*100

porcentaje_de_ganancia["p_2021_electronicos"] = (pivot_ganancia_por_categoria[("Electrónicos", 2021)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2021)]+ pivot_ganancia_por_categoria[("Juguetes", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2021)]+ pivot_ganancia_por_categoria[("Muebles", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]))*100

porcentaje_de_ganancia["p_2021_juguetes"] = (pivot_ganancia_por_categoria[("Juguetes", 2021)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2021)]+ pivot_ganancia_por_categoria[("Juguetes", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2021)]+ pivot_ganancia_por_categoria[("Muebles", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]))*100

porcentaje_de_ganancia["p_2021_libros"] = (pivot_ganancia_por_categoria[("Libros", 2021)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2021)]+ pivot_ganancia_por_categoria[("Juguetes", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2021)]+ pivot_ganancia_por_categoria[("Muebles", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]))*100

porcentaje_de_ganancia["p_2021_muebles"] = (pivot_ganancia_por_categoria[("Muebles", 2021)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos",2021)]+ pivot_ganancia_por_categoria[("Juguetes", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2021)]+ pivot_ganancia_por_categoria[("Muebles", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]))*100

porcentaje_de_ganancia["p_2021_deportes"] = (pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2021)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2021)]+ pivot_ganancia_por_categoria[("Juguetes", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2021)]+ pivot_ganancia_por_categoria[("Muebles", 2021)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2021)]))*100
                                                          

porcentaje_de_ganancia["p_2022_art_hogar"] = (pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2022)]+ pivot_ganancia_por_categoria[("Juguetes", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2022)]+ pivot_ganancia_por_categoria[("Muebles", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]))*100

porcentaje_de_ganancia["p_2022_electronicos"] = (pivot_ganancia_por_categoria[("Electrónicos", 2022)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2022)]+ pivot_ganancia_por_categoria[("Juguetes", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2022)]+ pivot_ganancia_por_categoria[("Muebles", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]))*100

porcentaje_de_ganancia["p_2022_juguetes"] = (pivot_ganancia_por_categoria[("Juguetes", 2022)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2022)]+ pivot_ganancia_por_categoria[("Juguetes", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2022)]+ pivot_ganancia_por_categoria[("Muebles", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]))*100

porcentaje_de_ganancia["p_2022_libros"] = (pivot_ganancia_por_categoria[("Libros", 2022)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2022)]+ pivot_ganancia_por_categoria[("Juguetes", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2022)]+ pivot_ganancia_por_categoria[("Muebles", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]))*100

porcentaje_de_ganancia["p_2022_muebles"] = (pivot_ganancia_por_categoria[("Muebles", 2022)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2022)]+ pivot_ganancia_por_categoria[("Juguetes", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2022)]+ pivot_ganancia_por_categoria[("Muebles", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]))*100
                                                                                          
porcentaje_de_ganancia["p_2022_deportes"] = (pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2022)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2022)]+ pivot_ganancia_por_categoria[("Juguetes", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2022)]+ pivot_ganancia_por_categoria[("Muebles", 2022)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2022)]))*100


porcentaje_de_ganancia["p_2023_art_hogar"] = (pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2023)]+ pivot_ganancia_por_categoria[("Juguetes", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2023)]+ pivot_ganancia_por_categoria[("Muebles", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]))*100

porcentaje_de_ganancia["p_2023_electronicos"] = (pivot_ganancia_por_categoria[("Electrónicos", 2023)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2023)]+ pivot_ganancia_por_categoria[("Juguetes", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2023)]+ pivot_ganancia_por_categoria[("Muebles", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]))*100

porcentaje_de_ganancia["p_2023_juguetes"] = (pivot_ganancia_por_categoria[("Juguetes", 2023)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2023)]+ pivot_ganancia_por_categoria[("Juguetes", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2023)]+ pivot_ganancia_por_categoria[("Muebles", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]))*100

porcentaje_de_ganancia["p_2023_libros"] = (pivot_ganancia_por_categoria[("Libros", 2023)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2023)]+ pivot_ganancia_por_categoria[("Juguetes", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2023)]+ pivot_ganancia_por_categoria[("Muebles", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]))*100

porcentaje_de_ganancia["p_2023_muebles"] = (pivot_ganancia_por_categoria[("Muebles", 2023)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2023)]+ pivot_ganancia_por_categoria[("Juguetes", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2023)]+ pivot_ganancia_por_categoria[("Muebles", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]))*100
                                                                                          
porcentaje_de_ganancia["p_2023_deportes"] = (pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]/(pivot_ganancia_por_categoria[("Artículos para el hogar", 2023)] + 
                                                                                          pivot_ganancia_por_categoria[("Electrónicos", 2023)]+ pivot_ganancia_por_categoria[("Juguetes", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Libros", 2023)]+ pivot_ganancia_por_categoria[("Muebles", 2023)]+ 
                                                                                          pivot_ganancia_por_categoria[("Deportes y diversión", 2023)]))*100

In [36]:
porcentaje_de_ganancia

,p_2020_art_hogar,p_2020_electronicos,p_2020_juguetes,p_2020_libros,p_2020_muebles,p_2020_deportes,p_2021_art_hogar,p_2021_electronicos,p_2021_juguetes,p_2021_libros,...,p_2022_juguetes,p_2022_libros,p_2022_muebles,p_2022_deportes,p_2023_art_hogar,p_2023_electronicos,p_2023_juguetes,p_2023_libros,p_2023_muebles,p_2023_deportes
Lugar de Compra,,,,,,,,,,,,,,,,,,,,,
Armenia,1.958360,86.483782,1.950511,0.975256,8.632091,0.000000,0.000000,74.696318,2.491631,0.000000,...,0.000000,8.309281,86.920857,0.000000,0.000000,86.070588,13.929412,0.000000,0.000000,0.000000
Barranquilla,1.567181,61.600395,1.098393,1.779151,31.034942,2.919938,2.528149,51.151184,2.672783,1.506883,...,3.803600,1.267675,52.671562,15.975234,4.911514,25.904564,8.683815,4.058895,32.256216,24.184997
Bogotá,2.241185,62.437900,2.537540,1.318051,26.514473,4.950850,1.932031,62.579721,2.743236,1.314031,...,2.785958,1.525563,28.433926,5.991190,2.192948,46.676396,3.836716,2.110548,37.139011,8.044381
Bucaramanga,0.387627,59.522335,5.440966,2.058744,25.188009,7.402319,0.552840,76.438403,1.860026,0.879518,...,9.011636,1.666712,27.593642,2.672977,3.021083,35.800908,2.050480,2.423294,36.289424,20.414810
Cali,1.727328,64.985127,2.427370,1.384552,22.486907,6.988716,1.621232,60.527359,2.343479,1.031539,...,2.901880,0.864727,26.756723,5.144423,1.334823,73.431272,1.315263,0.312368,20.516088,3.090187
Cartagena,4.135139,37.027088,5.033293,3.289952,42.816283,7.698245,0.913478,78.623240,3.862264,1.988816,...,5.515851,1.313609,16.433650,5.004225,3.148684,0.000000,5.884528,0.000000,63.460472,27.506316
Cúcuta,1.039922,72.143491,3.326927,0.333030,22.226468,0.930161,3.890015,47.874129,2.954796,2.536214,...,1.234187,0.721697,25.094190,1.166791,0.540221,67.889247,0.000000,0.636741,30.116976,0.816815
Inírida,0.000000,8.701172,0.000000,0.000000,0.000000,91.298828,0.000000,20.015030,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
Leticia,0.865830,45.815895,0.496763,0.409664,42.011559,10.400289,1.524332,62.107486,3.040112,1.092756,...,1.007069,1.532767,24.865659,5.764789,2.364114,70.171752,0.000000,0.000000,25.613255,1.850879


In [37]:
import plotly.graph_objects as go

tiendas_plot = porcentaje_de_ganancia.index.tolist()
anos_plot_nn = porcentaje_de_ganancia.columns.tolist()

# Crear una figura inicial con la primera tienda
fig = go.Figure()

# Agregar trazas para cada tienda
for local in porcentaje_de_ganancia.index:
    fig.add_trace(go.Bar(
        x=anos_plot_nn,
        y=porcentaje_de_ganancia.loc[local],  
        name=local,  # Nombre de la tienda
    ))

# Configurar el diseño del gráfico
fig.update_layout(
    title="Porcentaje de ganancia al año por Categoria",
    xaxis_title="Año",
    yaxis_title="% de ganancia",
    barmode= "group",
    legend_title="Tienda",
      
)

In [38]:
import plotly.express as px

fig = px.scatter_mapbox(utilidades, lat="lat", lon="lon",
                        color="Ganancia", size="Ganancia", size_max=30, zoom=5, mapbox_style="carto-positron",
                        hover_name="Lugar de Compra", hover_data="Ganancia", center={"lat":float(utilidades["lat"].mean()), "lon": float(utilidades["lon"].mean())},
                        title="Distribución geográfica de las ganancias")
fig.show()

C:\Users\herna\AppData\Local\Temp\ipykernel_23460\1545782739.py:3: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [39]:
utilidades

,Lugar de Compra,lat,lon,Precio,Costo de envío,Calificación,Ganancia,p_ganancia/precio,p_costo/precio
0,Bogotá,4.60971,-74.08175,1.814858e+09,96730000.0,4.113619,1.718128e+09,94.670107,5.329893
1,Medellín,6.25184,-75.56359,1.096669e+09,58502600.0,3.926534,1.038167e+09,94.665429,5.334571
2,Cali,3.43722,-76.52250,5.065261e+08,26913900.0,4.046673,4.796122e+08,94.686572,5.313428
3,Pereira,4.81333,-75.69611,2.259273e+08,12000300.0,4.013540,2.139270e+08,94.688424,5.311576
4,Barranquilla,10.96854,-74.78132,1.320405e+08,7034600.0,3.803226,1.250059e+08,94.672392,5.327608
5,Cartagena,10.39972,-75.51444,1.023767e+08,5478500.0,4.020661,9.689820e+07,94.648685,5.351315
6,Cúcuta,7.89391,-72.50782,9.936910e+07,5319100.0,3.940860,9.405000e+07,94.647129,5.352871
7,Bucaramanga,7.12539,-73.11980,9.057330e+07,4849000.0,3.977901,8.572430e+07,94.646325,5.353675
8,Riohacha,11.54444,-72.90722,7.387970e+07,3932800.0,3.895706,6.994690e+07,94.676752,5.323248
9,Leticia,-4.21528,-69.94056,6.714990e+07,3595100.0,3.922535,6.355480e+07,94.646157,5.353843


Finalmente para poder tomar una decision objetiva en base a cual de todas la sucursales debe cerrar se obtiene un indicador basado en la rentabilidad, las calificaciones obtenidas. El valor mas bajo deberá ser la sucursar que debera cerrar

In [40]:
# Calcular el total de ingresos y el promedio de calificaciones por año y por lugar de compra
ingresos_calificaciones = db_tiendas.groupby(["Lugar de Compra", "año"]).agg({
    "Precio": "sum",
    "Calificación": "mean"
}).reset_index().rename(columns={"Precio": "Total Ingresos", "Calificación": "Promedio Calificación"})

ingresos_calificaciones["Total Ingresos"].max()

np.float64(584437600.0)

In [41]:
ingresos_calificaciones["ingreso_normalizado"] = (ingresos_calificaciones["Total Ingresos"]-ingresos_calificaciones["Total Ingresos"].min())/(ingresos_calificaciones["Total Ingresos"].max()-ingresos_calificaciones["Total Ingresos"].min())
ingresos_calificaciones["calificacion_normalizada"] = (ingresos_calificaciones["Promedio Calificación"]-ingresos_calificaciones["Promedio Calificación"].min())/(ingresos_calificaciones["Promedio Calificación"].max()-ingresos_calificaciones["Promedio Calificación"].min())




In [42]:
ingresos_calificaciones

,Lugar de Compra,año,Total Ingresos,Promedio Calificación,ingreso_normalizado,calificacion_normalizada
0,Armenia,2020,5382100.0,3.111111,0.009001,0.190476
1,Armenia,2021,5500300.0,3.250000,0.009204,0.250000
2,Armenia,2022,1135400.0,4.000000,0.001734,0.571429
3,Armenia,2023,225100.0,5.000000,0.000176,1.000000
4,Barranquilla,2020,48804300.0,3.828283,0.083314,0.497835
...,...,...,...,...,...,...
70,Valledupar,2023,238600.0,5.000000,0.000199,1.000000
71,Villavicencio,2020,3482700.0,3.750000,0.005751,0.464286
72,Villavicencio,2021,2812400.0,4.000000,0.004604,0.571429
73,Villavicencio,2022,6916100.0,3.692308,0.011627,0.439560


In [43]:
ingresos_calificaciones["indicador"]= ingresos_calificaciones["ingreso_normalizado"] * 0.8 + ingresos_calificaciones["calificacion_normalizada"] * 0.2

In [44]:
ingresos_calificaciones

,Lugar de Compra,año,Total Ingresos,Promedio Calificación,ingreso_normalizado,calificacion_normalizada,indicador
0,Armenia,2020,5382100.0,3.111111,0.009001,0.190476,0.045296
1,Armenia,2021,5500300.0,3.250000,0.009204,0.250000,0.057363
2,Armenia,2022,1135400.0,4.000000,0.001734,0.571429,0.115673
3,Armenia,2023,225100.0,5.000000,0.000176,1.000000,0.200141
4,Barranquilla,2020,48804300.0,3.828283,0.083314,0.497835,0.166219
...,...,...,...,...,...,...,...
70,Valledupar,2023,238600.0,5.000000,0.000199,1.000000,0.200159
71,Villavicencio,2020,3482700.0,3.750000,0.005751,0.464286,0.097458
72,Villavicencio,2021,2812400.0,4.000000,0.004604,0.571429,0.117969
73,Villavicencio,2022,6916100.0,3.692308,0.011627,0.439560,0.097214


In [45]:
pivot_ingresos_calificaciones = ingresos_calificaciones.pivot(index='Lugar de Compra', columns=['año'], values='indicador').fillna(0)

In [ ]:
pivot_ingresos_calificaciones["promedio_indicador"] = (pivot_ingresos_calificaciones[2020] + pivot_ingresos_calificaciones[2021] + pivot_ingresos_calificaciones[2022] + pivot_ingresos_calificaciones[2023])/4



In [48]:
pivot_ingresos_calificaciones.sort_values(by="promedio_indicador", ascending=False, inplace=True)

In [49]:
pivot_ingresos_calificaciones

año,2020,2021,2022,2023,promedio_indicador
Lugar de Compra,,,,,
Bogotá,0.906475,0.920938,0.859444,0.294492,0.745337
Medellín,0.529907,0.636804,0.559687,0.209963,0.484090
Cali,0.340444,0.300880,0.336332,0.194579,0.293059
Pereira,0.206960,0.211605,0.208390,0.138230,0.191296
Cartagena,0.124400,0.159955,0.197063,0.105748,0.146791
Bucaramanga,0.151606,0.153828,0.147795,0.126219,0.144862
Barranquilla,0.166219,0.157523,0.136834,0.105623,0.141550
Valledupar,0.151415,0.052308,0.137793,0.200159,0.135419
Riohacha,0.148097,0.113601,0.142587,0.128848,0.133283


Luego de normalizado los ingresos y las calificaciones para poder poder compararlos con las calificaciones, se le otorgo un peso del 0.8 a los ingresos y un 0.2 a las calificaciones. Tras la creacion de este indicador obtenemos que la sucursal mas tentativa para que cierre sus puertas es **Manizales**